# Encontro DEVs

Código apresentado na segunda parte do encontro.

Exemplo de analise de sentimento através da classificação dos reviews retirados do IMDb sobre diversos filmes.

A idéia geral da estratégia, é fazer um indexamento das palavras para um número identificador (token), para então, serem usadas com entrada da rede neural. A indexação é feita pela frequencia que a palavra aparece em todo o conjunto de dados, por exemplo, o número 4 representa a 4ª palavra que mais aparece nos reviews.

Da mesma maneira, o mapeamento é feito para as palavras "Positivo" (1) ou "Negativo" (0), afim de indentificar sobre o sentimento exposto naquele review.

Como convenção, o "0" não representa nenhuma palavra, mas usada para representar alguma palavras desconhecida. Com isso, quando aparecer o "0", a análise sobre o sentimento de determinado review não é afetada.

### Ferramentas utilizadas:
<p>Python 3.6
<p>Tensorflow (pode ser o Theano também)
<p>Keras
<p>Numpy

In [1]:
import numpy as np
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence as prep
import time

Using TensorFlow backend.


In [2]:
# Carrega o conjunto de dados, mas mantem somente as n palavras mais mencionadas (relevantes)
# zerando o restante

# As 1000 palavras mais relevantes
top_words = 1000

# Carrega os conjunto de dados de reviews
# Cada review é classificado em Positivo (1) ou Negativo (0)
# Contem 25,000 reviews
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
print(len(X_train))
print(len(y_train))

25000
25000


In [7]:
# maximo de palavras que o review pode conter, preenchendo o restante com zero
max_words = 125
X_train = prep.pad_sequences(X_train, maxlen=max_words)
X_test = prep.pad_sequences(X_test, maxlen=max_words)
print(len(X_train))
print(len(X_test))

25000
25000


In [8]:
# Criando o modelo
model = Sequential()
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=10))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 125, 32)           32000     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 123, 32)           3104      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 61, 32)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 59, 64)            6208      
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 29, 64)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 27, 64)            12352     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 2, 64)             0         
__________

In [9]:
# Treinando a rede
start = time.time()
# Função de treinamento da rede. 
# O objetivo é aproximar uma função de acordo com os reviews e suas respectivas avaliações (Positivo ou Negativo)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=128, verbose=2)
print("Trainamento realizado em %.2f segundos." % (time.time() - start))
scores = model.evaluate(X_test, y_test, verbose=2)
print("Acurácia: %.2f%%" % (scores[1] * 100))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
 - 8s - loss: 0.6609 - acc: 0.5844 - val_loss: 0.6164 - val_acc: 0.6581
Epoch 2/5
 - 2s - loss: 0.5905 - acc: 0.6838 - val_loss: 0.6014 - val_acc: 0.6679
Epoch 3/5
 - 2s - loss: 0.5561 - acc: 0.7143 - val_loss: 0.6078 - val_acc: 0.6655
Epoch 4/5
 - 2s - loss: 0.5118 - acc: 0.7488 - val_loss: 0.6614 - val_acc: 0.6426
Epoch 5/5
 - 2s - loss: 0.4514 - acc: 0.7896 - val_loss: 0.7090 - val_acc: 0.6267
Trainamento realizado em  16.52 segundos.
Acurácia: 62.67%


In [10]:
# Pega o dicionário da palavra para cada índice
word_to_id = keras.datasets.imdb.get_word_index()
# Criar outro discionário a partir do anterior que mapaeia o índice a respectiva palavra
id_to_word = {value:key for key,value in word_to_id.items()}

In [12]:
# Parte de teste da rede neural
# A partir de um novo review, o mapeamento para os respectivos índices de cada palavra
# Após o mapeamento, o review pode ser submetivo para a rede para a análise

#my_review = 'worse movie bad script i hate the actors and photography'
my_review = 'good movie very well written and i love it'
words = my_review.split(" ")
print(words)

my_r = {key:value for key, value in id_to_word.items() if value in words}
print(my_r.keys())
my_rr = np.array(list(my_r.keys()))
print(my_rr.size)

embedding_matrix = np.zeros((1, max_words)).astype(int)
embedding_matrix[0, 0:my_rr.size] = my_rr
embedding_matrix

['good', 'movie', 'very', 'well', 'written', 'and', 'i', 'love', 'it']
dict_keys([10, 9, 52, 395, 70, 116, 2, 17, 49])
9


array([[ 10,   9,  52, 395,  70, 116,   2,  17,  49,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0]])

In [13]:
score = model.predict(embedding_matrix)
if (score[0] * 100) > 50.00:
    print("Good")
    print("Accuracy: %.2f%%" % (score[0] * 100))
else:
    print("Bad")
    print("Accuracy: %.2f%%" % ((1-score[0]) * 100))

Good
Accuracy: 79.70%
